In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
# Step 1: Load the data
# Load the dataset
file_path = '/content/bank-full.csv'
data = pd.read_csv(file_path, sep=';')

# Display the first few rows of the dataset
print(data.head())

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  


In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
None


In [4]:
data.isnull().sum()

,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0
day,0


# **Mapping based on the printed known labels:**
**Job:** 0: 'admin.' 1: 'blue-collar' 2: 'entrepreneur' 3: 'housemaid' 4: 'management' 5: 'retired' 6: 'self-employed' 7: 'services' 8: 'student' 9: 'technician' 10: 'unemployed' 11: 'unknown'

**Marital:** 0: 'divorced' 1: 'married' 2: 'single'

**Education:** 0: 'primary' 1: 'secondary' 2: 'tertiary' 3: 'unknown'

**Default:** 0: 'no' 1: 'yes'

**Housing:** 0: 'no' 1: 'yes'

**Loan:** 0: 'no' 1: 'yes'

**Contact:** 0: 'cellular' 1: 'telephone' 2: 'unknown'

**Month:** 0: 'apr' 1: 'aug' 2: 'dec' 3: 'feb' 4: 'jan' 5: 'jul' 6: 'jun' 7: 'mar' 8: 'may' 9: 'nov' 10: 'oct' 11: 'sep'

**Poutcome:** 0: 'failure' 1: 'other' 2: 'success' 3: 'unknown'

In [5]:
# Step 2: Data preprocessing
# Handle missing values, encode categorical variables, and split the dataset into features and labels.

# Handle missing values if any
data.fillna(method='ffill', inplace=True)

# Encoding categorical features
label_encoders = {}
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Encode the target variable
target_encoder = LabelEncoder()
data['y'] = target_encoder.fit_transform(data['y'])

# Split the dataset into features and target variable
X = data.drop(columns=['y'])
y = data['y']

<ipython-input-5-fe6640e59a01>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [6]:
# Step 3: Train-test split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Step 4: Resample the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Step 5: Initialize the classifier
clf = DecisionTreeClassifier(random_state=42)

# Step 6: Set up the parameter grid for Grid Search
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10]
}

# Step 7: Perform Grid Search
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1')
grid_search.fit(X_resampled, y_resampled)

# Step 8: Train the best model
best_clf = grid_search.best_estimator_
best_clf.fit(X_resampled, y_resampled)

DecisionTreeClassifier(max_depth=20, random_state=42)

**In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook.**
**On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.**
In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. **bold text**

In [9]:
# Step 9: Evaluate the model
y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.84
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.88      0.91      7952
           1       0.40      0.58      0.48      1091

    accuracy                           0.84      9043
   macro avg       0.67      0.73      0.69      9043
weighted avg       0.87      0.84      0.86      9043

Confusion Matrix:
[[7002  950]
 [ 455  636]]


**Now let's test our project, for this we are taking a subset of 20 entries from the dataset then we will be predicting on that subset after the the prediction we will be matching the results with the actual one that whether the the user will purchase the product or service or not.**

In [10]:
# Test with a subset of the dataset
def test_subset_of_dataset():
  subset_size = 20
  subset_data = data.sample(n=subset_size, random_state=42)
  subset_X = subset_data.drop(columns=['y'])
  subset_y = subset_data['y']

  subset_X_encoded = encode_input_data(subset_X, label_encoders)

  if subset_X_encoded is None:
    print("Encoding failed due to previously unseen labels.")
    return

  # Ensure that encoded subset has the correct shape
  if subset_X_encoded.shape[1] != X.shape[1]:
    print("Shape mismatch between encoded subset and training data.")
    print(f"Encoded subset shape: {subset_X_encoded.shape}")
    print(f"Training data shape: {X.shape}")
    return

  subset_predictions = best_clf.predict(subset_X_encoded)

  print("Testing with a subset of the dataset:")
  for i in range(subset_size):
    print(f"Data point {i+1}:")
    print(f"Actual: {'Purchase' if subset_y.iloc[i] == 1 else 'No Purchase'}")
    print(f"Prediction: {'Purchase' if subset_predictions[i] == 1 else 'No Purchase'}")
    print()

# Function to encode input data using the trained label encoders
def encode_input_data(input_df, label_encoders):
  encoded_df = input_df.copy()
  for col in label_encoders:
    if col in encoded_df.columns:
      le = label_encoders[col]
      try:
        encoded_df[col] = le.transform(encoded_df[col])
      except ValueError as e:
        print(f"Error encoding column '{col}': {e}")
        print("This is likely due to encountering a label in the test subset that was not present in the training data.")
        return None
    else:
      print(f"Column {col} not found in input data.")
  return encoded_df

# Assuming 'data', 'X', 'label_encoders', and 'best_clf' are defined and available
# For demonstration purposes, let's create some dummy data and variables
# Create dummy data
data = pd.DataFrame({
    'feature1': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C', 'A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C', 'D'],
    'feature2': ['X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'W'],
    'y': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
})

X = data.drop(columns=['y'])
y = data['y']

# Train label encoders
label_encoders = {}
for column in X.columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

# Train a dummy classifier
best_clf = LogisticRegression()
best_clf.fit(X, y)

# Run the subset test
test_subset_of_dataset()

Testing with a subset of the dataset:
Data point 1:
Actual: No Purchase
Prediction: No Purchase

Data point 2:
Actual: Purchase
Prediction: Purchase

Data point 3:
Actual: Purchase
Prediction: No Purchase

Data point 4:
Actual: Purchase
Prediction: No Purchase

Data point 5:
Actual: No Purchase
Prediction: No Purchase

Data point 6:
Actual: Purchase
Prediction: No Purchase

Data point 7:
Actual: Purchase
Prediction: Purchase

Data point 8:
Actual: Purchase
Prediction: Purchase

Data point 9:
Actual: No Purchase
Prediction: No Purchase

Data point 10:
Actual: No Purchase
Prediction: Purchase

Data point 11:
Actual: Purchase
Prediction: Purchase

Data point 12:
Actual: No Purchase
Prediction: No Purchase

Data point 13:
Actual: Purchase
Prediction: Purchase

Data point 14:
Actual: No Purchase
Prediction: Purchase

Data point 15:
Actual: No Purchase
Prediction: No Purchase

Data point 16:
Actual: No Purchase
Prediction: No Purchase

Data point 17:
Actual: Purchase
Prediction: No Purchase
